In [4]:
import os
os.getcwd()
os.chdir("..")
os.getcwd()

'/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System'

In [8]:
%%capture
!pip install -r requirement.txt

In [54]:
import re
import chromadb
from llama_index.readers.file import PyMuPDFReader
from llama_index.core import SimpleDirectoryReader , VectorStoreIndex , StorageContext, get_response_synthesizer
from llama_index.core.node_parser import SentenceSplitter, SentenceWindowNodeParser ,SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.vector_stores import MetadataFilters, MetadataFilter
from llama_index.llms.ollama import Ollama
from llama_index.core.query_engine import RetrieverQueryEngine


In [30]:
loading_mapping = {'.pdf': PyMuPDFReader()}

documents = SimpleDirectoryReader('./data',file_extractor = loading_mapping).load_data()

In [32]:
len(documents)

2147

In [46]:
current_section = "unknown"
for doc in documents:
    match = re.search(r"(\d{4})", doc.metadata.get("file_name",""))
    if match :
        doc.metadata['year'] = int(match.group(0))
        doc.metadata['company'] = "coca-cola"
        doc.metadata['filings']= "10-k"

    text = doc.text.replace("\n"," ").replace("  "," ")
    match = re.search(r"(ITEM\s+\d+[A-Z]?\.\s+[A-Z\s,&\-]+)",text)
    if match :
        current_section = match.group(0).strip()
        doc.metadata['section']=current_section
        continue
    doc.metadata['section']=current_section
        

In [48]:
documents[12].metadata

{'file_path': '/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System/data/2015-cocacola-10k-filing.pdf',
 'file_name': '2015-cocacola-10k-filing.pdf',
 'file_type': 'application/pdf',
 'file_size': 1717600,
 'creation_date': '2025-08-31',
 'last_modified_date': '2025-05-23',
 'total_pages': 216,
 'source': '13',
 'year': 2015,
 'company': 'coca-cola',
 'filings': '10-k',
 'section': 'ITEM 1.  BUSINESS I'}

In [50]:
node_parser = SentenceSplitter(chunk_size =2048)

nodes = node_parser.get_nodes_from_documents(documents)

In [52]:
len(nodes)

2146

In [62]:
embed_model = HuggingFaceEmbedding(model_name ="BAAI/bge-base-en-v1.5")

db = chromadb.PersistentClient(".db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection = chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [64]:
index = VectorStoreIndex(nodes,storage_context=storage_context,embed_model=embed_model)

In [66]:
filters = MetadataFilters(
    filters=[
        MetadataFilter(key="year",value=2015,operator = ">="),
        MetadataFilter(key="year",value=2025,operator = "<=")
    ]
)

In [68]:
vector_retriever = index.as_retriever(
    similarity_top_k =5,
    filters=filters
)

In [96]:
query = "What is the sale in 2014 and 2015 ?"
ans = vector_retriever.retrieve(query)

In [84]:
len(ans)

5

In [98]:
for i in ans:
    print(i.metadata)
    print("")
    print(i.text)
    print("--"*100)

{'file_path': '/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System/data/2015-cocacola-10k-filing.pdf', 'file_name': '2015-cocacola-10k-filing.pdf', 'file_type': 'application/pdf', 'file_size': 1717600, 'creation_date': '2025-08-31', 'last_modified_date': '2025-05-23', 'total_pages': 216, 'source': '29', 'year': 2015, 'company': 'coca-cola', 'filings': '10-k', 'section': 'ITEM 5. MARKET FOR REGISTRANT'}

The following table presents information with respect to purchases of common stock of the Company made during the three months ended December 31, 2014, by the Company or any "affiliated
purchaser" of the Company as defined in Rule 10b-18(a)(3) under the Exchange Act.
Period
Total Number of
Shares Purchased1
 
Average
Price Paid
Per Share
Total Number of
Shares Purchased
as Part of Publicly
Announced Plan 2
Maximum Number of
Shares That May
Yet Be Purchased
Under the Publicly
Announced Plan
September 27, 2014 through October 24, 2014
5,150,833
 $
42.03
5,150,833
345,20

In [102]:
from llama_index.retrievers.bm25 import BM25Retriever

In [138]:
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)
db = chromadb.PersistentClient(path=".db")
chroma_collection = db.get_or_create_collection("dense_vectors")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(
    docstore=docstore, vector_store=vector_store
)
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context,embed_model=embed_model)

In [160]:
bm25_retriever = BM25Retriever.from_defaults(docstore=index.docstore,similarity_top_k=5)

TypeError: BM25Retriever.from_defaults() got an unexpected keyword argument 'filters'

In [168]:
ans = bm25_retriever.retrieve("what are finance policy statemnets in 2020?")

In [170]:
for i in ans:
    print(i.metadata)
    print(i.text)

{'file_path': '/Users/nikhil/Documents/GitHub_portfolio/1-Enterprise-Grade RAG System/data/2021-cocacola-10k-filing.pdf', 'file_name': '2021-cocacola-10k-filing.pdf', 'file_type': 'application/pdf', 'file_size': 1523345, 'creation_date': '2025-08-17', 'last_modified_date': '2025-05-23', 'total_pages': 194, 'source': '59', 'year': 2021, 'company': 'coca-cola', 'filings': '10-k', 'section': 'ITEM 6. INTENTIONALLY OMITTED ITEM'}
Total capital expenditures for property, plant and equipment and the percentage of such totals by operating segment and Corporate were as follows (in millions):
Year Ended December 31,
2020
2019
Capital expenditures
$
1,177 
$
2,054 
Europe, Middle East & Africa
2.3 %
5.2 %
Latin America
0.5 
6.8 
North America
15.5 
19.1 
Asia Pacific
1.7 
2.3 
Global Ventures
22.2 
10.2 
Bottling Investments
40.3 
40.7 
Corporate
17.6 
15.7 
Cash Flows from Financing Activities
Net cash provided by (used in) financing activities is summarized as follows (in millions):
Year Ended